## Data Synthesis using Generative Adversarial Network (Not Used)

### Data Preparation

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Load dataset normal
df = pd.read_csv("SP1_new.csv")

# Pilih fitur yang akan digunakan untuk sintesis data fault
features = ['Output Frequency', 'Output Current', 'Output Power', 'Fin Temperature', 'Cooling Fin Temperature']
data = df[features].values

# Normalisasi data agar sesuai untuk Neural Network
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# Konversi ke Tensor
data_tensor = torch.FloatTensor(data_normalized)
dataset = DataLoader(data_tensor, batch_size=64, shuffle=True)


### Generator

In [5]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)


### Diskriminator

In [7]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid() 
        )

    def forward(self, x):
        return self.model(x)


### Model and Optimizer Init

In [8]:
# Parameter model
input_dim = 5  # Jumlah fitur
latent_dim = 10  # Dimensi noise (input ke Generator)
lr = 0.0002  # Learning rate

# Inisialisasi model
generator = Generator(latent_dim, input_dim)
discriminator = Discriminator(input_dim)

# Optimizer
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

# Loss function
criterion = nn.BCELoss()


### Data Training

In [9]:
# Hyperparameters
num_epochs = 5000
batch_size = 64

for epoch in range(num_epochs):
    for real_data in dataset:
        batch_size = real_data.size(0)
        
        # ======== TRAIN DISCRIMINATOR ========
        real_labels = torch.ones(batch_size, 1)  # Label 1 untuk data asli
        fake_labels = torch.zeros(batch_size, 1)  # Label 0 untuk data buatan
        
        optimizer_D.zero_grad()
        
        # Evaluasi data asli
        real_output = discriminator(real_data)
        loss_real = criterion(real_output, real_labels)
        
        # Evaluasi data sintetik
        noise = torch.randn(batch_size, latent_dim)
        fake_data = generator(noise)
        fake_output = discriminator(fake_data.detach())
        loss_fake = criterion(fake_output, fake_labels)
        
        # Total loss Discriminator
        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizer_D.step()

        # ======== TRAIN GENERATOR ========
        optimizer_G.zero_grad()
        
        fake_output = discriminator(fake_data)
        loss_G = criterion(fake_output, real_labels)  # Generator ingin fool Discriminator
        loss_G.backward()
        optimizer_G.step()

    if epoch % 500 == 0:
        print(f"Epoch [{epoch}/{num_epochs}] | Loss_D: {loss_D:.4f} | Loss_G: {loss_G:.4f}")

print("Training GAN selesai!")


Epoch [0/5000] | Loss_D: 1.3516 | Loss_G: 0.5986
Epoch [500/5000] | Loss_D: 1.0097 | Loss_G: 1.1501
Epoch [1000/5000] | Loss_D: 1.1532 | Loss_G: 0.8698
Epoch [1500/5000] | Loss_D: 1.5330 | Loss_G: 0.6314
Epoch [2000/5000] | Loss_D: 1.3269 | Loss_G: 0.6982
Epoch [2500/5000] | Loss_D: 1.0819 | Loss_G: 0.9170
Epoch [3000/5000] | Loss_D: 0.8628 | Loss_G: 1.4938
Epoch [3500/5000] | Loss_D: 1.0117 | Loss_G: 1.6976
Epoch [4000/5000] | Loss_D: 0.7498 | Loss_G: 1.3781
Epoch [4500/5000] | Loss_D: 0.9112 | Loss_G: 1.2428
Training GAN selesai!


### Data Synthesis

In [11]:
# Buat 1000 sampel data fault sintetis
num_samples = 1000
noise = torch.randn(num_samples, latent_dim)
synthetic_fault_data = generator(noise).detach().numpy()

# Transform back ke skala asli
synthetic_fault_data = scaler.inverse_transform(synthetic_fault_data)

# Simpan ke CSV
df_synthetic = pd.DataFrame(synthetic_fault_data, columns=features)
df_synthetic.to_csv("data_vfd_fault_synthetic.csv", index=False)

print("Data fault sintetis berhasil dibuat dan disimpan!")


Data fault sintetis berhasil dibuat dan disimpan!
